In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBoostRegressor

In [ ]:
target_map = data['sales'].to_dict()

data['lag1'] = (data['date'] - pd.Timedelta('364 days')).map(target_map)
data['lag2'] = (data['date'] - pd.Timedelta('728 days')).map(target_map)
data['lag3'] = (data['date'] - pd.Timedelta('1092 days')).map(target_map)